Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.3 MB/s eta 0:00:00


Cirq ile Bell Durumları

In [2]:
import cirq

# Qubitler
q0, q1 = cirq.LineQubit.range(2)

def bell_state_circuit(state_type="phi_plus"):
    circuit = cirq.Circuit()
    circuit.append(cirq.H(q0))
    circuit.append(cirq.CNOT(q0, q1))

    if state_type == "phi_minus":
        circuit.append(cirq.Z(q0))   # Faz farkı ekle
    elif state_type == "psi_plus":
        circuit.append(cirq.X(q1))   # İkinci qubit ters çevir
    elif state_type == "psi_minus":
        circuit.append([cirq.X(q1), cirq.Z(q0)])

    return circuit

sim = cirq.Simulator()
for state in ["phi_plus", "phi_minus", "psi_plus", "psi_minus"]:
    circuit = bell_state_circuit(state)
    result = sim.simulate(circuit)
    print(f"\nBell State |{state}>:")
    print(circuit)
    print(result.final_state_vector)



Bell State |phi_plus>:
0: ───H───@───
          │
1: ───────X───
[0.70710677+0.j 0.        +0.j 0.        +0.j 0.70710677+0.j]

Bell State |phi_minus>:
0: ───H───@───Z───
          │
1: ───────X───────
[ 0.70710677+0.j  0.        +0.j -0.        +0.j -0.70710677+0.j]

Bell State |psi_plus>:
0: ───H───@───────
          │
1: ───────X───X───
[0.        +0.j 0.70710677+0.j 0.70710677+0.j 0.        +0.j]

Bell State |psi_minus>:
0: ───H───@───Z───
          │
1: ───────X───X───
[ 0.        +0.j  0.70710677+0.j -0.70710677+0.j -0.        +0.j]


Qiskit ile Bell Durumları

In [3]:
from qiskit import QuantumCircuit, Aer, execute

def bell_state_qiskit(state_type="phi_plus"):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)

    if state_type == "phi_minus":
        qc.z(0)
    elif state_type == "psi_plus":
        qc.x(1)
    elif state_type == "psi_minus":
        qc.x(1)
        qc.z(0)

    return qc

simulator = Aer.get_backend('statevector_simulator')
for state in ["phi_plus", "phi_minus", "psi_plus", "psi_minus"]:
    qc = bell_state_qiskit(state)
    result = execute(qc, simulator).result()
    statevector = result.get_statevector()
    print(f"\nBell State |{state}>:")
    print(qc)
    print(statevector)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Bell Durumları

In [4]:
from qutip import basis, tensor, hadamard_transform, qeye, cnot

# |00> başlangıç durumu
q0 = basis(2,0)
q1 = basis(2,0)
psi0 = tensor(q0, q1)

# Hadamard ilk qubit üzerinde
H = tensor(hadamard_transform(1), qeye(2))
psi1 = H * psi0

# CNOT uygula
CNOT = cnot()
phi_plus = CNOT * psi1  # |Φ+>

# Diğer Bell durumları
phi_minus = (tensor(qeye(2), qeye(2)) * phi_plus).unit()
phi_minus = (tensor(qeye(2), qeye(2)) * phi_plus) # sadece Z uygulayacağız
phi_minus = (tensor(qeye(2), qeye(2)) * phi_plus)

phi_minus = (tensor(qeye(2), qeye(2)) * phi_plus)

phi_minus = (tensor(qeye(2), qeye(2)) * phi_plus)

# Alternatif daha kısa yol:
phi_minus = (tensor(qeye(2), qeye(2)) * phi_plus)  # üstüne Pauli-Z uygulanmalı
psi_plus  = (tensor(qeye(2), qeye(2)) * phi_plus)  # üstüne Pauli-X uygulanmalı
psi_minus = (tensor(qeye(2), qeye(2)) * phi_plus)  # X + Z kombinasyonu

print("Bell State |Φ+>:\n", phi_plus)


ImportError: cannot import name 'hadamard_transform' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

In [5]:
# QuTiP’de aslında hazır fonksiyon da var:

from qutip.states import bell_state
print(bell_state('00'))  # |Φ+>
print(bell_state('01'))  # |Φ->
print(bell_state('10'))  # |Ψ+>
print(bell_state('11'))  # |Ψ->

ModuleNotFoundError: No module named 'qutip.states'